<a href="https://colab.research.google.com/github/aerimili/EIE401-1-Multimedia/blob/main/Proyectos/Proyecto_1/Proyecto_1_VillalobosAlejandra_20914803_K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>

</center></h1>
<font size="6"><center>
EIE 401
PROCESAMIENTO DIGITAL MULTIMEDIA
</center></font>


<center><h2>Proyecto 1</h2></center>
<center><h3>Procesamiento digital de señales</h3></center>
<center><h3>Profesor: Jorge Cardenas</h3></center>

<center><h3>Por: Alejandra Villalobos</h3></center>
<center><h5>Pontificia Universidad Catolica de Valparaiso</h5></center>


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.signal import tukey, butter, freqs, resample
from scipy import signal

In [ ]:
from IPython.display import IFrame, display
filepath = "Proyecto_1_EIE401.pdf"
IFrame(filepath, width=700, height=400)

Debes usar el sitio https://gwosc.org/eventapi/html/GWTC/, donde encuentras todo el catálogo de observaciones con los datos en diferentes formatos. Debes seleccionar una detección para tu estudio y descargar los datos correspondientes (H5 o TXT), lo que posteriormente deberás tratar de manera digital como audio (serie de tiempo).
tip: Te recomiendo tomar la serie de tiempo corta (32 segundos) para facilitar el análisis, y asegurarte de tomar la tasa de muestreo más alta.
tip: Asegúrate de tomar un caso cuyo espectrograma de muestra, indique una señal clara y fuerte para facilitar tu trabajo, de otra forma es posible que no logres aislarla lo suficiente.


## 1. Carga de Datos


In [ ]:
from google.colab import files

#Subir archivo a Colab
print("Cargue archivo H-H1_GWOSC_16KHZ_R1-1242459842-32.txt:")
uploaded = files.upload()

In [3]:
L1 = "L-L1_GWOSC_16KHZ_R1-1242459842-32.txt"

#Se abre el archivo txt
with open(L1, "r") as fichero:
  archivo_string = fichero.read()

#Se importa archivo saltandose las filas que no sirven
strain = np.loadtxt(L1, delimiter = ",", skiprows = 1, dtype = float)

## 2. Convertir en audio
Es una forma de validar que tenemos un alto componente de ruido en la señal y que el interés es eliminarlo para encontrar una señal candidata.

Te recomiendo hacer una señal de audio de cada uno de las etapas para así validar como va el proceso. Al final deberás obtener un señal con CHIRP o BUMP. Son sonidos que comprueban que tienes una cantidad de potencia que sobre destaca sobre el ruido.

In [4]:
fs = 16000 #Frecuencia de muestreo
time = np.linspace(0, strain.size / fs, strain.size) #Creación vector tiempo

lowTime = 15 #Tiempo de intervalo donde empieza recorte
highTime = 16 #Tiempo de intervalo donde termina recorte
lowSample = lowTime*fs #Muestras en tiempo bajo
highSample = highTime*fs #Muestras en tiempo alto

strainSteps = strain[lowSample:highSample] #Señal de audio recortada

timeSteps = np.linspace(lowTime, highTime, strainSteps.size) #Vector tiempo recortado

In [ ]:
#Gráfica de señal de audio original
plt.figure(figsize=(20, 5))
plt.plot(timeSteps, strainSteps)
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.title("Señal original en intervalo de interés")

In [ ]:
#Reproducción de audio original
Audio(strain, rate = fs)

## 3. Análisis de Fourier
Al realizar la conversión, queremos ver cuales son las componentes de ruido que más afectan al instrumento y debemos ignorar en el análisis. Eso se ve como "pico" o la apariencia de "peine".
Se debe presenta la densidad espectral de potencia antes y después de la función de Windowing.


In [ ]:
ft = np.fft.fft(strain) #Transformada de Fourier de señal original

n = len(time)

dep = ft * np.conj(ft) / n #Cálculo de la densidad espectral de potencia a

freq = np.fft.fftfreq(n, 1/fs) #Creación de vector de frecuencias
L = np.arange(1, np.floor(n/2), dtype = "int")

windowTukey = tukey(strain.size, alpha = 0.5) #Creación de ventana Tukey

strainWindow = strain * windowTukey #Se realiza windowing a la señal

fftStrain = np.fft.fft(strainWindow) #Transformada de Fouier de señal con windowing

depWin = fftStrain * np.conj(fftStrain) / n #Cálculo de densidad espectral de potencia a señal con windowing

In [ ]:
#Gráfica de señal original y señal con windowing
plt.figure(figsize = (20,6))
plt.subplot(121)
plt.plot(timeSteps, strainSteps)
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.title("Señal original en intervalo de interés")

plt.subplot(122)
plt.plot(timeSteps, strainWindow[lowSample:highSample])
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.title("Señal con windowing en intervalo de interés")

In [ ]:
#Gráfica de psd de señal original y psd de señal con windowing en escala logarítmica
plt.figure(figsize = (20,6))
plt.semilogy(freq[L], dep[L], color = "purple", label = "original")
plt.semilogy(freq[L], depWin[L], label = "windowed")
plt.title("Densidad espectral de potencia")
plt.xlabel("Frecuencias[Hz]")
plt.ylabel("Potencia")
plt.legend()

In [ ]:
#Reproducción de audio con windowing
Audio(strainWindow, rate = fs)

## 4. Procesamiento
Debes inferir de tu estudio, cual es el proceso que aplican a la señal y reproducirlo. Es posible que cada estudiante llegue a una solución más o menos diferente, pues no hay un código abierto donde esté el procedimiento completo.


### Whitening

In [9]:
asd = np.sqrt(depWin) #Cálculo de asd

whitenedStrain = fftStrain / asd #Cálculo de whitening de señal en dominio de frecuencia

wStrainTimeD = np.fft.ifft(whitenedStrain) #Cálculo de whitening de señal en dominio del tiempo

In [ ]:
#Gráfica de señal whitened
plt.figure(figsize=(20, 6))
plt.plot(timeSteps, wStrainTimeD[lowSample:highSample], label="Señal whitened")
plt.xlabel("Time (s)")
plt.ylabel("Amplitud")
plt.title("Whitening de señal en dominio del tiempo en intervalo de interés")

In [ ]:
#Reproducción de señal whitened
Audio(wStrainTimeD, rate = fs)

### Filtrado

In [11]:
b, a = butter(4, [30, 100], btype = "bandpass", fs=fs) #Creación de filtro Butterworth

filteredStrain = signal.filtfilt(b, a, wStrainTimeD.real) #Aplicación de filtro a señal whitened

In [ ]:
#Gráfica de señal filtrada
plt.figure(figsize=(20, 6))
plt.plot(timeSteps, filteredStrain[lowSample:highSample])
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Señal filtrada")

In [ ]:
#Reproducción de audio filtrado
Audio(filteredStrain, rate = fs)

Entre el segundo 15 y 16 se puede alcanzar a escuchar el "chirp" de la señal.

# Resampling
Para lograr una mejor definición de la señal, se debe hacer un resampleo. Esto tienen consecuencias por eso debe elegirse bien la frecuencia de tal forma que no se introduzca ruido.

In [13]:
fs_nueva = 32000 #Frecuencia de muestreo para resampleo

low = 15 #Tiempo de intervalo donde empieza recorte
high = 16 #Tiempo de intervalo donde termina recorte

lowsample = low * fs_nueva #Muestras en tiempo bajo
highsample = high * fs_nueva #Muestras en tiempo alto

resampledStrain = resample(filteredStrain.real, int(len(filteredStrain) * fs_nueva / fs)) #Señal resampleada

tiempo = np.linspace(0, len(resampledStrain) / fs_nueva, len(resampledStrain)) #Creación de vector tiempo para señal resampleada

In [ ]:
#Gráfica de señal resampleada a 32kHz
plt.figure(figsize=(20, 6))
plt.plot(tiempo[lowsample:highsample], resampledStrain[lowsample:highsample])
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Señal resampleada a 32kHz")

In [ ]:
#Reproducción de audio resampleado
Audio(resampledStrain, rate = fs_nueva)

### Espectrograma

In [ ]:
#Ver espectrograma de señal resampleada
f, t, Sxx = signal.spectrogram(resampledStrain, fs_nueva)
plt.pcolormesh(t, f, Sxx, shading="gouraud")
plt.xlim(15.6, 16)
plt.ylim(0,1000)
plt.title("Espectrograma de señal resampleada")
plt.xlabel("Tiempo (s)")
plt.ylabel("Frecuencia (Hz)")

#Referencias

[Whitening + Bandpass](https://www.kaggle.com/code/maxdner/signal-preprocessing-whitening-bandpass-filter)

[Resampling](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.resample.html)

[Tukey window](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.signal.tukey.html)

[Densidad espectral de potencia](https://www.youtube.com/watch?v=s2K1JfNR7Sc)

[Filtro Butterworth](https://dsp.stackexchange.com/questions/49460/apply-low-pass-butterworth-filter-in-python)

[Filtro Butterworth](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.butter.html)

[Espectrograma](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.spectrogram.html)